In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
# import tensorflow as tf

In [0]:
num_classes = 200
input_size = 200  
hidden_size = 200 
batch_size = 1   
sequence_length = 1  
num_layers = 1  


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size, batch_first=True)

  def forward(self, hidden, x):
        
    x = x.view(batch_size, sequence_length, input_size)

        
    out, hidden = self.rnn(x, hidden)
    return hidden, out.view(-1, num_classes)

  def init_hidden(self):
        
    return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [0]:
my_vecs=[]
inputs = Variable(torch.Tensor(vectors))
model = Model()
hidden = model.init_hidden()
for input in (inputs):
      
  hidden, output = model(hidden, input)
  my_vecs.append(output)

vecs=[]
for item in my_vecs:

  x=list(hidden.detach().numpy()[0][0])
  y=list(item.detach().numpy()[0])
  vec=(x+y)
  vecs.append(vec)


In [0]:

layers = []
layers.append(nn.Linear(400, 64))
layers.append(nn.ReLU())
layers.append(nn.Linear(64, 64))
layers.append(nn.ReLU())

layers.append(nn.Linear(64,1))

net = nn.Sequential(*layers)

In [0]:
for item in vecs:
  i = torch.from_numpy(np.array(item))
  score=net(i)
  print(score)

In [0]:
class Listwise(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Listwise, self).__init__();
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
    self.mlp = nn.Sequential(
        nn.Linear(2*hidden_size, 64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )
  
  def forward(self, input_doc_vectors):
    outputs = []
    for i, doc_vec in enumerate(input_doc_vectors):
      print("doc vec shape: ", doc_vec.shape)
      doc_vec_in = doc_vec.view(1,1,doc_vec.shape[0])
      print("doc vec in shape: ", doc_vec_in.shape)
      if i == 0:
        output, h_next = self.rnn(doc_vec_in)
      else:
        output, h_next = self.rnn(doc_vec_in, h_next)
      outputs.append(output.view(output.shape[2]))
    
    final_hidden_state = h_next.view(h_next.shape[2])
    print("final hidden state shape: ", final_hidden_state.shape)
    doc_vec_scores = []

    for out in outputs:
      mlp_in = torch.cat([out, final_hidden_state]).unsqueeze(0)
      doc_vec_score = self.mlp(mlp_in)[0]
      doc_vec_scores.append(doc_vec_score)
    
    doc_vec_scores_tensor = torch.stack(doc_vec_scores)
    return doc_vec_scores_tensor
  
  def train(self, X, y, optimizer, epochs=10, batch_size=20, device='cpu'):
    for ep in range(epochs):
      print("epoch no: ", ep)
      losses = []
      for i, (doc_vec_set, target) in enumerate(zip(X, y)):
        doc_vec_tensor_set = [torch.from_numpy(x).float().to(device) for x in doc_vec_set]
        target_tensor = torch.tensor(target).float().to(device)
        predicted_scores_tensor = self(doc_vec_tensor_set)
        
        loss = nn.MSELoss()(predicted_scores_tensor, target_tensor)
        losses.append(loss)
        if (i+1) % batch_size == 0:
          optimizer.zero_grad()
          mean_loss = torch.stack(losses).mean()
          mean_loss.backward()
          optimizer.step()
          losses = []
  
  def inference(self, X):
    for doc_vec_set in X:
      doc_vec_tensor_set = [torch.from_numpy(x).float().to(device) for x in doc_vec_set]
      predicted_scores_tensor = self(doc_vec_tensor_set)
      predicted_order_of_documents = predicted_scores_tensor.argsort()
      return predicted_order_of_documents.cpu().numpy()


In [0]:
def load_pickle(path):
  with open(path, "rb") as f:
    data = pickle.load(f)
  return data

def save_pickle(path, data):
  with open(path, "wb") as f:
    pickle.dump(data, f)

In [0]:
path_to_ir = "/content/drive/My Drive/IR Project DATA"

In [0]:
dataset = load_pickle(os.path.join(path_to_ir, "Train_data"))

In [ ]:
X, y = dataset
print(X[0])
print(y[0])

In [0]:
input_size=2
hidden_size=2
model = Listwise(input_size=input_size, hidden_size=hidden_size)

In [ ]:
x1 = [np.array([1.,2.]), np.array([1.,3.]), np.array([2.,6.])]
y1 = [2.,1.,3.]
model.train([x1], [y1], optim.Adam(model.parameters(), lr=0.001), batch_size=1)